In [53]:
import re, subprocess, pyodbc , os
import numpy as np # linear algebra 
from sqlalchemy import func, create_engine
from config import mongoConn, connstr  
import pandas as pd 

In [95]:
engine = create_engine(connstr)
conn = engine.connect()
query = """ \
SELECT QPK, QGROUP, QuestionTypeCode, ROW_NUMBER() OVER(ORDER BY IDTEXT ASC) -1 AS Row 
FROM  vw_MetricsCompositeKeys
WHERE PK_Form LIKE '%EINS%' 
ORDER BY IDTEXT ASC 
"""  
df = pd.read_sql(query,con=conn) 
conn.close()
df 

,QPK,QGROUP,QuestionTypeCode,Row
0,20850,2110,NCEinsteinPublicIP,0
1,20861,2111,NCEinsteinUnannounced,1
2,20881,2113,NCEinsteinTaps,2
3,20891,2114,NCEinsteinVLAN,3
4,20900,2115,NCEinsteinUnmonitoredTraffic,4
5,20940,2120,NCEinsteinNATInfo,5
6,20950,2121,NCEinsteinBGP,6
7,20960,2122,NCEinsteinArtifactGrid,7


In [96]:
dirr= r'D:\dev\CyberScope\CyberScopeBranch\CSwebdev\code\CyberScope\EINSTEIN\2021' 
templates = []
ffilter = '2021_A_EINSTEIN_0' 
files = [f for f in os.scandir(dirr) if ffilter in f.path ]    
for file in files:  
    with open(file.path, encoding="utf-8", errors='ignore', mode='r' ) as f: 
        templates.append({'d':f'{dirr}\\{file.name}', 'c':f.read() })   

In [97]:
for i, r in df.iterrows():
    if i > 6:
        for t in templates: 
            dd = t['d'].replace('_0',f'_{i+10}') 
            cc = t['c'].replace('_0',f'_{i+10}') 
            cc = cc.replace('NCEinsteinPublicIP',f'EinsteinPublicIP') 
            cc = cc.replace('EinsteinPublicIP',r['QuestionTypeCode']) 
            cc = cc.replace('2110',str(r['QGROUP'])) 
            cc = cc.replace('20850',str(r['QPK'])) 
            print( dd )
            print( cc )
            #with open(dd, 'w') as writer:
            #    writer.write(cc)
 

D:\dev\CyberScope\CyberScopeBranch\CSwebdev\code\CyberScope\EINSTEIN\2021\2021_A_EINSTEIN_17.aspx
﻿<%@ Page Title="" Language="vb" AutoEventWireup="false" MasterPageFile="~/Form.Master" 
    CodeBehind="2021_A_EINSTEIN_17.aspx.vb" Inherits="CyberScope._2021_A_EINSTEIN_17" %>

<%@ Register Src="../../CustomControls/CBlabel.ascx" TagName="CBlabel" TagPrefix="uc1" %>
<%@ Register Src="../../CustomControls/CBoptionalLinks.ascx" TagName="CBoptionalLinks" TagPrefix="uc2" %>
<%@ Register Src="../../CustomControls/NCEinsteinArtifactGrid.ascx" TagName="NCEinsteinArtifactGrid" TagPrefix="uc3" %>
 
<asp:Content ID="Content1" ContentPlaceHolderID="ContentPlaceHolder1" runat="Server"> 
    <asp:Panel ID="Panel1" runat="server"> 
        <table class="table table-sm"> 
			<tr>
                <td colspan="6" class="SectionHead">
                    <uc1:CBlabel ID="CBlabelGroup" runat="server" LabelType="Group" PK_key="2122" />
                </td>
            </tr> 
			<tr>
                <td sty

,QPK,QGROUP,QuestionTypeCode,Row#
0,20850,2110,NCEinsteinPublicIP,0
1,20861,2111,NCEinsteinUnannounced,1
2,20881,2113,NCEinsteinTaps,2
3,20891,2114,NCEinsteinVLAN,3
4,20900,2115,NCEinsteinUnmonitoredTraffic,4
5,20940,2120,NCEinsteinNATInfo,5
6,20960,2122,NCEinsteinBGP,6
7,20950,2121,NCARTIFACT,7


In [ ]:
engine = create_engine(connstr)
conn = engine.connect()
query = """ \
SELECT QPK, QGROUP, IdText, QuestionTypeCode , QuestionTypeCode, QTEXT, help_text   
FROM  vw_MetricsCompositeKeys
WHERE PK_Form LIKE '%EINS%'
"""  
df = pd.read_sql(query,con=conn) 
conn.close()

In [ ]:
sections = [ 
{'1': 'Public IP Address Specification'}
{'2': 'Unannounced IP Address Specification'}
{'3': 'EINSTEIN Test Access Point (TAP) Locations'}
{'4': 'Virtual LANs (VLANs) Specification'}
{'5': 'Unmonitored Internet Traffic'}
{'6': 'Network Address Translation (NAT) Specification'}
{'7': 'Border Gateway Protocol (BGP) Specification'}
{'8': 'Network Diagrams'}        
]


In [11]:
def getip(): 
    return f"{np.random.randint(1, 255)}.{np.random.randint(1, 255)}.{np.random.randint(1, 255)}.{np.random.randint(1, 255)}"
getip()

'70.233.135.81'

In [16]:
engine = create_engine(connstr)
conn = engine.connect()
query = """ \
SELECT TOP 1 * FROM EinsteinPublicIP 
"""  
df = pd.read_sql(query,con=conn)  
df.drop('PK_EinsteinPublicIP', axis=1, inplace=True)
df.drop('PK_OrgSubmission', axis=1, inplace=True)
df.drop('UserId', axis=1, inplace=True)
df.drop('isActive', axis=1, inplace=True)
df.drop('DateDeactivated', axis=1, inplace=True)
for c in df.columns:
    print(f"d['{c}'] = '{getip()}'"  )

 

d['Name'] = getip()
d['ISP'] = getip()
d['OtherISP'] = getip()
d['ExternalOrg'] = getip()
d['Visible'] = getip()
d['StartingIP'] = getip()
d['EndingIP'] = getip()
d['TIC_MTIPS'] = getip()


In [36]:
lst = []
ln = 20000
for i in range(ln):
    d={}
    d['StartingIP'] = getip()
    d['EndingIP'] = getip()    
    d['Name_Purpose'] = getip()
    d['ISP'] = 'IBM' 
    d['OtherISP'] = 'Other'
    d['External_Organization'] = fake.address()[:25]
    d['TIC_MTIPS'] = 'TIC'
    d['Visible'] = 'Yes' 
    lst.append(d)
df=pd.DataFrame(lst)
df.to_excel(fr'c:\temp\EinsteinPublicIP_{ln}.xlsx', index=False)    
